# Basic Quasar Demo: GHZ and $|W_N\rangle$ Circuits

## Circuit 1: GHZ Circuit
This demo shows how to use quasar to produce an $N=4$ GHZ state $(|0000\rangle + |1111\rangle) / \sqrt(2)$ and to perform 1- and 2-body Pauli expectation values for this state. This covers most of the day-to-day basics of quasar, including interactions with the `Circuit` and `Gate` classes, and construction, simulation, and expectation value observation of quantum circuits.

### Import
First, import numpy and quasar:

In [1]:
import numpy as np
import quasar

### Circuit Construction
Now build the circuit. We always start with an empty quasar `Circuit` object with a known number of qubits $N$, arranged in a line. We then add quantum gates to the circuit with the `add_gate` function. `add_gate` takes positional kwargs of `T` (moment in time in the circuit, starting at 0) and `key` (single qubit index or tuple of qubit indices). `add_gate` also requires the `gate` kwarg which takes a quasar `Gate` object. Common `Gate` objects can be obtained as static attributes of the `Gate` class (for parameter-free gates like the `I` and `CNOT` gates seen here) or by calling static methods of the `Gate` class to provide the initial values of required paramters (for parameter-based gates like `Ry(theta)`). 

In this case, we start an empty `Circuit` object with $N=4$. We then add a Hadamard gate `H` to bring qubit 0 into the even superposition state $(|0\rangle + |1\rangle)/\sqrt{2}$. Next, we add a chain of 3x controlled not gates `CNOT` to entangle all 4 qubits into the GHZ state $(|0000\rangle + |1111\rangle) / \sqrt(2)$. Finally, we `print` the circuit, which draws a simple ASCII diagram of the circuit:

In [2]:
circuit = quasar.Circuit(N=4)
circuit.add_gate(T=0, key=0, gate=quasar.Gate.H)
circuit.add_gate(T=1, key=(0,1), gate=quasar.Gate.CNOT)
circuit.add_gate(T=2, key=(1,2), gate=quasar.Gate.CNOT)
circuit.add_gate(T=3, key=(2,3), gate=quasar.Gate.CNOT)
print(circuit)

T   : |0|1|2|3|
               
|0> : -H-@-----
         |     
|1> : ---X-@---
           |   
|2> : -----X-@-
             | 
|3> : -------X-

T   : |0|1|2|3|


In [3]:
print('N: %d' % circuit.N)
print('nmoment: %d' % circuit.nmoment)
print('ngate: %d' % circuit.ngate)
print('ngate1: %d' % circuit.ngate1)
print('ngate2: %d' % circuit.ngate2)

N: 4
nmoment: 4
ngate: 4
ngate1: 1
ngate2: 3


### Simulation
Now we simulate the propagation of the state vector through the circuit, using the `simulate` method:

In [4]:
wfn = circuit.simulate()
print(wfn)
print(wfn.dtype)

[0.70710678+0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]
complex128


State vector simulation defaults to `np.complex128` precision (double complex), but we can override this to a real type if we know that all gate operations are real [$U(2^N)$ vs $O(2^N)$] or if we want reduced precision, by invoking the `dtype` kwarg:

In [5]:
wfn = circuit.simulate(dtype=np.float32)
print(wfn)
print(wfn.dtype)

[0.70710677 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.70710677]
float32


/Users/parrish/Code/quasar/quasar/quasar.py:1533: ComplexWarning: Casting complex values to real discards the imaginary part
  U=np.array(gate.U, dtype=dtype),
/Users/parrish/Code/quasar/quasar/quasar.py:1540: ComplexWarning: Casting complex values to real discards the imaginary part
  U=np.array(gate.U, dtype=dtype),


By default, circuit simulation starts from the reference state $|00\ldots \rangle$. We can also provide a custom starting state vector through the `wfn` kwarg:

In [6]:
wfn0 = np.zeros((2**circuit.N), dtype=np.complex128)
wfn0[0] = 1.0 # The reference ket |0000>
wfn = circuit.simulate(wfn=wfn0)
print(wfn)

[0.70710678+0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]


### Detailed Simulation Steps
Sometimes it can really help debugging to see the state vector through time during circuit evolution. To this end, the `simulate_steps` method yields the time moment and state vector after each time moment in the circuit. Note that the yielded `wfn` array is a buffer owned by `simulate_steps` to avoid repeated memory allocations in deep circuit simulations - for long term storage of the wavefunction state vector history, this value should be copied. `simulate_steps` takes the same arguments as `simulate`: 

In [7]:
for T, wfn in circuit.simulate_steps(dtype=np.float64):
        print('Time: %d\n' % T)
        print(wfn)

Time: 0

[0.70710678 0.         0.         0.         0.         0.
 0.         0.         0.70710678 0.         0.         0.
 0.         0.         0.         0.        ]
Time: 1

[0.70710678 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.70710678 0.         0.         0.        ]
Time: 2

[0.70710678 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.70710678 0.        ]
Time: 3

[0.70710678 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.70710678]


### Quasar Qubit Ordering
Looking at the details of the simulation steps above, we can see that the state vector after the Hadamard gate at time 0 on qubit 0 corresponded to a coefficient of $+1/\sqrt{2}$ in the 0-th and 8-th positions of the state vector, corresponding to a state of $(|0000\rangle + |1000\rangle) / \sqrt{2}$. This is the standard QIS ordering of Nielson and Chuang - this ordering is used in, e.g., Cirq, but the opposite ordering is also sometimes seen, e.g., in Qiskit. For explicit definition of our ordering convention, consider the quasar CNOT gate: 

In [8]:
print(quasar.Gate.CNOT.U)

[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]]


### 1-Body Density Matrices and Pauli Expectation Values
As a quantum chemist, the state vector is a highly compelling object, but it cannot ever be observed. Instead, we must content ourselves with observable quantities such as low-order density matrices. Utility routines are provided in quasar to compute these by contractions of the wavefunction, i.e., assuming infinite statistical convergence of observations. The 1-particle (transition) density matrix (1PDM) is defined as $D_{pq}^{A} = \langle\Psi_1 | \left (|p_A\rangle \langle|q_A |\right) | \Psi_2 \rangle$. This can be computed for a given state vector or pair of state vectors by the `compute_1pdm` static method of `Circuit`:

In [9]:
DA = quasar.Circuit.compute_1pdm(wfn1=wfn, wfn2=wfn, A=0)
print(DA)

[[0.5 0. ]
 [0.  0.5]]


Often in QIS, one works in the more-familiar ansatz of Pauli expectation values restricted to a diagonal (non-transition) case. To facilitate this, we also provide the `compute_pauli_1` static method of `Circuit`. This returns a list of real Pauli expectation values corresponding to $\langle \Psi | \hat O_A | \Psi \rangle$ for operators $\hat O_A \in [\hat I_A, \hat X_A, \hat Y_A, \hat Z_A]$:

In [10]:
PA = quasar.Circuit.compute_pauli_1(wfn=wfn, A=0)
print(PA)

[1. 0. 0. 0.]


Both of these observations indicate that the GHZ state has $\langle\hat Z_0\rangle = 0$. Repeating for $A>0$ would show the same for all qubits.

### 2-Body Density Matrices and Pauli Expectation Values
Similarly, we can define the 2-particle (transition) density matrix (2PDM) as $D_{pqrs}^{AB} = \langle\Psi_1 | \left (|p_A\rangle \langle|q_A \otimes |r_B \rangle \langle s_B| \right) | \Psi_2 \rangle$. This can be computed for a given state vector or pair of state vectors by the `compute_2pdm` static method of `Circuit`, and is returned with $|p_A\rangle \otimes |r_B\rangle$ on the rows and $\langle q_A | \otimes \langle s_B|$ on the columns:

In [11]:
DAB = quasar.Circuit.compute_2pdm(wfn1=wfn, wfn2=wfn, A=0, B=1)
print(DAB)

[[0.5 0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0.5]]


Similarly, we can define the 2-body Pauli expectation vmalues restricted to a diagonal (non-transition) case. $\langle \Psi | \hat O_A \otimes \hat P_B | \Psi \rangle$ for operators $\hat O_A \in [\hat I_A, \hat X_A, \hat Y_A, \hat Z_A]$ (in rows) and $\hat P_B \in [\hat I_B, \hat X_B, \hat Y_B, \hat Z_B]$ (in columns):

In [12]:
P = circuit.compute_pauli_2(wfn=wfn, A=0, B=1)
print(P)

[[1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 1.]]


This study shows the remarkable property of the GHZ state: looking at the observations any one qubit, we are equally likely to get $|0\rangle$ or $|1\rangle$ ($\langle \hat Z_A \rangle = 0$), but if we look at the simultaneous observations of any two qubits, they are perfectly positively correlated ($\langle Z_A \otimes Z_B \rangle = 1$). 

## Circuit 2: $|W_N\rangle$ Circuit
This demo shows how to construct a $|W_N\rangle$ state $(|1000\rangle + |0100\rangle + |0010\rangle + |0001\rangle) / 2$, and demonstrates interacting with circuit parameters. We first construct our circuit, making noteworthy use of a number of "controlled F" (CF) gates, each of which requires an input parameter $\theta$:
\begin{equation}
\hat U_{\mathrm{CF}} (\theta)
\equiv
\left [
\begin{array}{cccc}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & +\cos(\theta) & +\sin(\theta) \\
0 & 0 & +\sin(\theta) & -\cos(\theta) \\
\end{array}
\right ]
\end{equation}
The circuit is:

In [13]:
A = np.arccos(0.5)
B = np.arccos(0.5 / np.sin(A))
C = np.arccos(0.5 / (np.sin(A) * np.sin(B)))
circuit = quasar.Circuit(N=4)
circuit.add_gate(T=0, key=0, gate=quasar.Gate.X)
circuit.add_gate(T=1, key=(0,1), gate=quasar.Gate.CF(theta=A))
circuit.add_gate(T=2, key=(1,2), gate=quasar.Gate.CF(theta=B))
circuit.add_gate(T=3, key=(2,3), gate=quasar.Gate.CF(theta=C))
circuit.add_gate(T=4, key=(3,2), gate=quasar.Gate.CNOT)
circuit.add_gate(T=5, key=(3,1), gate=quasar.Gate.CNOT)
circuit.add_gate(T=6, key=(2,1), gate=quasar.Gate.CNOT)
circuit.add_gate(T=7, key=(3,0), gate=quasar.Gate.CNOT)
circuit.add_gate(T=8, key=(2,0), gate=quasar.Gate.CNOT)
circuit.add_gate(T=9, key=(1,0), gate=quasar.Gate.CNOT)
print(circuit)

T   : |0|1|2|3|4|5|6|7|8|9|
                           
|0> : -X-@-----------X-X-X-
         |           | | | 
|1> : ---F-@-----X-X-|-|-@-
           |     | | | |   
|2> : -----F-@-X-|-@-|-@---
             | | |   |     
|3> : -------F-@-@---@-----

T   : |0|1|2|3|4|5|6|7|8|9|


The resultant state vector is:

In [14]:
print(circuit.simulate())

[0. +0.j 0.5+0.j 0.5+0.j 0. +0.j 0.5+0.j 0. +0.j 0. +0.j 0. +0.j 0.5+0.j
 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j]


This provides the "canonical" $|W_N\rangle$ state, with same-amplitude configurations. However, by tweaking the parameters of the CF gates, we can easily produce configuration interaction singles (CIS) states with variable-amplitude configurations (see https://arxiv.org/pdf/1901.01234.pdf for details). To accomplish, there are a number of utility methods to access the parameters of a circuit. Note that all param entries are always sorted lexically according to param_keys [tuple of `(T, key, param_name_in_gate)`]:

In [15]:
print(circuit.nparam)
print(circuit.param_keys)
print(circuit.param_values)
print(circuit.params)

3
[(1, (0, 1), 'theta'), (2, (1, 2), 'theta'), (3, (2, 3), 'theta')]
[1.0471975511965976, 0.9553166181245092, 0.7853981633974481]
OrderedDict([((1, (0, 1), 'theta'), 1.0471975511965976), ((2, (1, 2), 'theta'), 0.9553166181245092), ((3, (2, 3), 'theta'), 0.7853981633974481)])


We also provide a utility string to print a summary of the parameter values:

In [16]:
print(circuit.param_str)

T     Qubits     Name       Gate      :                    Value
1     (0, 1)     theta      CF        :   1.0471975511965976E+00
2     (1, 2)     theta      CF        :   9.5531661812450919E-01
3     (2, 3)     theta      CF        :   7.8539816339744806E-01



Before tweaking parameters, we must point out that `Gate` and `Circuit` objects are *mutable* with respect to parameter values. Therefore, we often want to make a copy of these objects before tweaking parameters. This can be accomplished by the `Gate` or `Circuit` copy methods:

In [17]:
circuit2 = circuit.copy()

One way to set circuit parameters is by providing a dictionary of key -> value pairs (most useful for tweaking only a subset of parameters):

In [18]:
circuit2.set_params({ (1, (0, 1), 'theta') : 1.0 })

This affects the parameters of `circuit2`:

In [19]:
print(circuit2.param_str)

T     Qubits     Name       Gate      :                    Value
1     (0, 1)     theta      CF        :   1.0000000000000000E+00
2     (1, 2)     theta      CF        :   9.5531661812450919E-01
3     (2, 3)     theta      CF        :   7.8539816339744806E-01



But not `circuit`:

In [20]:
print(circuit.param_str)

T     Qubits     Name       Gate      :                    Value
1     (0, 1)     theta      CF        :   1.0471975511965976E+00
2     (1, 2)     theta      CF        :   9.5531661812450919E-01
3     (2, 3)     theta      CF        :   7.8539816339744806E-01



Another way to set circuit parameters is to provide the values of all parameters in an iterable with the same order as `param_keys` (useful in optimizations, in which all parameters are usually treated as an ordered array):

In [21]:
circuit2.set_param_values([1.0]*3)
print(circuit2.param_str)

T     Qubits     Name       Gate      :                    Value
1     (0, 1)     theta      CF        :   1.0000000000000000E+00
2     (1, 2)     theta      CF        :   1.0000000000000000E+00
3     (2, 3)     theta      CF        :   1.0000000000000000E+00



This now provides for variable-amplitude CIS states:

In [22]:
print(circuit2.simulate())

[0.        +0.j 0.59582324+0.j 0.3825737 +0.j 0.        +0.j
 0.45464871+0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.54030231+0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j]


## Advanced Features
Side note: it is often useful to access and inspect individual gates:

In [23]:
CF01 = circuit2.gate(T=1, key=(0,1))
print(CF01.params)
print(CF01.U)

OrderedDict([('theta', 1.0)])
[[ 1.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j]
 [ 0.        +0.j  1.        +0.j  0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.54030231+0.j  0.84147098+0.j]
 [ 0.        +0.j  0.        +0.j  0.84147098+0.j -0.54030231+0.j]]


And for advanced users, it can be useful to know about the following `Circuit` attributes:

In [24]:
print(circuit.gates)

OrderedDict([((0, (0,)), <quasar.quasar.Gate object at 0x111413160>), ((1, (0, 1)), <quasar.quasar.Gate object at 0x11141a780>), ((2, (1, 2)), <quasar.quasar.Gate object at 0x11141a748>), ((3, (2, 3)), <quasar.quasar.Gate object at 0x11126c780>), ((4, (3, 2)), <quasar.quasar.Gate object at 0x111413390>), ((5, (3, 1)), <quasar.quasar.Gate object at 0x111413390>), ((6, (2, 1)), <quasar.quasar.Gate object at 0x111413390>), ((7, (3, 0)), <quasar.quasar.Gate object at 0x111413390>), ((8, (2, 0)), <quasar.quasar.Gate object at 0x111413390>), ((9, (1, 0)), <quasar.quasar.Gate object at 0x111413390>)])


In [25]:
print(circuit.Ts)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [26]:
print(circuit.TAs)

{(7, 3), (9, 1), (8, 0), (2, 1), (6, 2), (5, 1), (9, 0), (3, 3), (2, 2), (1, 1), (3, 2), (0, 0), (8, 2), (4, 2), (1, 0), (5, 3), (7, 0), (6, 1), (4, 3)}
